<a href="https://colab.research.google.com/github/imrohu/NLP/blob/master/NLP2Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')
import os



Mounted at /content/gdrive


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras 
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

url='https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
data=pd.read_csv(url, sep='\t', header=0, quoting=3)

data.head(100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
95,96,3,", I suspect ,",2
96,97,3,"I suspect ,",2
97,98,3,I suspect,2
98,99,3,I,2


In [30]:
data.shape

(156060, 4)

In [31]:
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
pd.set_option('max_colwidth',300)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
from keras.utils import to_categorical

X = data.cleaned_reviews.values
Y = to_categorical(data.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer(max_features=3000,stop_words = None, ngram_range=(1,2))
X = vectorizer.fit_transform(X)



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2003)

In [0]:
import numpy as np



x_train_np = x_train.toarray()
y_train_np = np.array(y_train)

x_test_np = x_test.toarray()
y_test_np = np.array(y_test)

In [0]:
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

In [0]:

import torch
torch.manual_seed(2020)
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu, softmax, sigmoid
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import SGD, Adam
from torch.nn import L1Loss, CrossEntropyLoss

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()

def cnn(data_x):
  
  model.add(Conv1D(filters = 128, kernel_size=3, activation='relu', input_shape=(data_x.shape[1],1)))
  
  model.add(MaxPooling1D(pool_size =2))
 
  model.add(Conv1D(filters = 256, kernel_size=3, activation='relu'))
  
  model.add(MaxPooling1D(pool_size =2))
  model.add(Flatten())
  
  model.add(Dense(100, activation='relu'))
  
  model.add(Dropout(0.2))
  
  model.add(Dense(5, activation='softmax')) 






In [0]:
from keras import metrics
from sklearn.metrics import f1_score

In [0]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:

cnn(x_train_np)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_m,precision_m, recall_m])


In [0]:
history = model.fit(x_train, y_train_np, epochs=10, batch_size=256)


Epoch 1/10
109242/109242 [==============================] - 42s 388us/step - loss: 1.0905 - categorical_accuracy: 0.5681 - f1_m: 0.4811 - precision_m: 0.6566 - recall_m: 0.3850
Epoch 2/10
109242/109242 [==============================] - 41s 380us/step - loss: 0.9550 - categorical_accuracy: 0.6238 - f1_m: 0.5739 - precision_m: 0.6946 - recall_m: 0.4895
Epoch 3/10
109242/109242 [==============================] - 42s 382us/step - loss: 0.8940 - categorical_accuracy: 0.6486 - f1_m: 0.6136 - precision_m: 0.7076 - recall_m: 0.5420
Epoch 4/10
109242/109242 [==============================] - 42s 382us/step - loss: 0.8452 - categorical_accuracy: 0.6674 - f1_m: 0.6409 - precision_m: 0.7172 - recall_m: 0.5796
Epoch 5/10
 67072/109242 [=================>............] - ETA: 16s - loss: 0.7934 - categorical_accuracy: 0.6880 - f1_m: 0.6672 - precision_m: 0.7298 - recall_m: 0.6148

In [0]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

In [0]:
model.save("NLP2.pt")

In [0]:

from numpy import loadtxt
from keras.models import load_model
 
load_model('NLP2.pt', custom_objects={'f1_m': f1_m, 'precision_m':precision_m, 'recall_m':recall_m})

#model.summary()
